In [38]:
from spacerocks.spacerock import SpaceRock, RockCollection, Origin
from spacerocks.observing import Observatory, DetectionCatalog
from spacerocks.spice import SpiceKernel
from spacerocks.time import Time
from spacerocks.nbody import Simulation, Integrator, Force

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np

kernel = SpiceKernel()
kernel.load("/Users/kjnapier/data/spice/latest_leapseconds.tls")
kernel.load("/Users/kjnapier/data/spice/de440s.bsp")

In [39]:
origin = Origin.ssb()

In [40]:
ARCSEC_PER_RAD = 206264.80624709636

In [41]:
epoch = Time.now()
planets_names = ["sun", "mercury barycenter"]
planets = [SpaceRock.from_spice(name, epoch, frame="ECLIPJ2000", origin=origin) for name in planets_names]

In [48]:
sim = Simulation()

sim.set_epoch(epoch)
sim.set_frame("ECLIPJ2000")
sim.set_origin(origin)
sim.set_integrator(Integrator.ias15(timestep=2.0))
sim.add_force(Force.newtonian_gravity()) 
sim.add_force(Force.solar_gr())

print(sim.origin)

for planet in planets:
    sim.add(planet)
    
sim.move_to_center_of_mass()

Origin: SSB with mu = 0.00029630927493457475


In [49]:
mercury = sim.particles[1]
mercury.calculate_orbit()

evec_0 = np.array(mercury.evec) / np.linalg.norm(mercury.evec)

In [50]:
evec_0

array([0.21924234, 0.973856  , 0.05947506])

In [51]:
n = 10
sim.integrate(epoch + n * 100 * 365.25)

In [52]:
mercury = sim.particles[1]
mercury.calculate_orbit()

evec_1 = np.array(mercury.evec) / np.linalg.norm(mercury.evec)

In [53]:
precession_rate = np.arccos(np.dot(evec_0, evec_1)) * ARCSEC_PER_RAD / n

print(f"Mercury's pericenter precession rate due to GR is {precession_rate:.2f} arcsec/century.")

Mercury's pericenter precession rate due to GR is 43.01 arcsec/century.
